In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, glob
import numpy as np
from tqdm import tqdm
import torch
import smplx
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import gaussian_filter1d
import json
import csv
import pdb
import pickle

### original egobody 

In [5]:
# with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_00040.npz','rb') as f:
with open("/home/yuxinyao/datasets/egobody/smplx_camera_wearer/recording_20210907_S02_S01_01/body_idx_1/results/frame_01551/000.pkl",'rb') as f:
# with open("/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_00000.npz",'rb') as f:
    temp = dict(pickle.load(f))
    # temp = dict(np.load(f))

In [6]:
temp_bm = smplx.create('/home/yuxinyao/body_models/', model_type='smplx',
                            gender=str(temp['gender']), ext='npz',
                            create_body_pose=True,
                            create_betas=True,
                            create_left_hand_pose=True,
                            create_right_hand_pose=True,
                            create_expression=True,
                            create_jaw_pose=True,
                            create_leye_pose=True,
                            create_reye_pose=True,
                            create_transl=True,
                            batch_size=1
                            ).cuda()

In [ ]:
temp.keys()

dict_keys(['transf_rotmat', 'transf_transl', 'trans', 'poses', 'betas', 'gender', 'mocap_framerate', 'global_orient', 'joints', 'marker_cmu_41', 'marker_ssm2_67'])

In [7]:
# temp.keys()
# temp_params = {}
# temp_params['poses'] = torch.FloatTensor(temp['poses'][0,:]).unsqueeze(0).cuda()
# temp_params['betas'] = torch.FloatTensor(temp['betas']).cuda()
# temp_params['transl'] = torch.FloatTensor(temp['trans'][0,:]).unsqueeze(0).cuda()
# temp_params['global_orient'] = torch.FloatTensor(temp['global_orient'][0]).unsqueeze(0).cuda()
temp_params = {}
for key in temp.keys():
    if key in ['pose_embedding', 'camera_rotation', 'camera_translation', 'gender']:
        continue
    else:
        temp_params[key] = torch.tensor(temp[key]).cuda()

In [26]:
for k,v in temp_params.items():
    print("{} shape : {}".format(k,v.shape))

poses shape : torch.Size([1, 20, 153])
betas shape : torch.Size([16])
transl shape : torch.Size([1, 20, 3])
global_orient shape : torch.Size([1, 20, 3])


In [9]:
b = temp_bm(**temp_params)

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [1, 12]->[1, 1, 12] [6, 45]->[1, 45, 6]

In [ ]:
import trimesh
temp_t = trimesh.Trimesh(vertices=b.vertices[0].squeeze(0).detach().cpu().numpy(), faces=temp_bm.faces)
temp_t.export('/home/yuxinyao/datasets/egobody/test/subseq_115.obj')

'# https://github.com/mikedh/trimesh\nv -0.03735602 -0.55491805 -0.25487053\nv -0.04150860 -0.55404961 -0.25455916\nv -0.04209381 -0.55534351 -0.25508779\nv -0.03774866 -0.55616283 -0.25520265\nv -0.04712424 -0.55140364 -0.26663616\nv -0.04834477 -0.55588579 -0.26470411\nv -0.04802595 -0.55853200 -0.26547131\nv -0.04722165 -0.55730021 -0.26794344\nv 0.00886586 -0.49973521 -0.28690654\nv 0.02382400 -0.49791577 -0.28781819\nv 0.02296103 -0.49095288 -0.28179240\nv 0.00855053 -0.49043617 -0.27936876\nv -0.01485464 -0.47569832 -0.25787091\nv -0.00889027 -0.48091719 -0.26469439\nv -0.00721562 -0.47045276 -0.26337084\nv -0.01862103 -0.46951136 -0.25330690\nv -0.03538990 -0.64407611 -0.19691674\nv -0.04111569 -0.63996148 -0.20285511\nv -0.04090903 -0.63593948 -0.19699973\nv 0.07935655 -0.56315124 -0.22600040\nv 0.08296362 -0.56283033 -0.22376859\nv 0.08355048 -0.56420684 -0.22400731\nv 0.07969001 -0.56444728 -0.22615251\nv 0.09383881 -0.56134856 -0.23176424\nv 0.09341812 -0.56588733 -0.2296342

### CMU 


In [11]:
# with open('/home/yuxinyao/datasets/egobody/canicalized-camera-wearer/er/recording_20210910_S06_S05_01/subseq_00040.npz','rb') as f:
# with open("/home/yuxinyao/datasets/egobody/smplx_camera_wearer/recording_20210907_S02_S01_01/body_idx_1/results/frame_01551/000.pkl",'rb') as f:
with open("/home/yuxinyao/datasets/AMASS-Canonicalized-locomotion-MPx2/data/CMU/subseq_56360.npz",'rb') as f:
    # temp = dict(pickle.load(f))
    temp_cmu = dict(np.load(f))

In [12]:
temp_cmu['poses'].shape

(20, 156)

In [20]:
cmu_bm = smplx.create('/home/yuxinyao/body_models/', model_type='smplx',
                            gender=str(temp_cmu['gender']), ext='npz',
                            create_body_pose=True,
                            create_betas=True,
                            create_left_hand_pose=True,
                            create_right_hand_pose=True,
                            create_expression=True,
                            create_jaw_pose=True,
                            create_leye_pose=True,
                            create_reye_pose=True,
                            create_transl=True,
                            batch_size=20
                            ).cuda()

In [14]:
temp_cmu.keys()

dict_keys(['transf_rotmat', 'transf_transl', 'trans', 'poses', 'betas', 'gender', 'mocap_framerate', 'joints', 'marker_cmu_41', 'marker_ssm2_67'])

In [22]:
cmu_params = {}
cmu_params['poses'] = torch.FloatTensor(temp_cmu['poses'][:,3:]).cuda()
cmu_params['betas'] = torch.FloatTensor(temp_cmu['betas']).repeat(20,1).cuda()
cmu_params['transl'] = torch.FloatTensor(temp_cmu['trans']).cuda()
cmu_params['global_orient'] = torch.FloatTensor(temp_cmu['poses'][:,:3]).cuda()


In [23]:
cmu_params['global_orient'].shape

torch.Size([20, 3])

In [24]:
for k,v in cmu_params.items():
    print("{} shape : {}".format(k,v.shape))

poses shape : torch.Size([20, 153])
betas shape : torch.Size([20, 16])
transl shape : torch.Size([20, 3])
global_orient shape : torch.Size([20, 3])


In [25]:
c = cmu_bm(**cmu_params)

RuntimeError: einsum(): operands do not broadcast with remapped shapes [original->remapped]: [20, 26]->[20, 1, 1, 26] [10475, 3, 20]->[1, 10475, 3, 20]

In [ ]:
import trimesh
temp_t = trimesh.Trimesh(vertices=b.vertices[0].squeeze(0).detach().cpu().numpy(), faces=temp_bm.faces)
temp_t.export('/home/yuxinyao/datasets/egobody/test/subseq_115.obj')

'# https://github.com/mikedh/trimesh\nv -0.03735602 -0.55491805 -0.25487053\nv -0.04150860 -0.55404961 -0.25455916\nv -0.04209381 -0.55534351 -0.25508779\nv -0.03774866 -0.55616283 -0.25520265\nv -0.04712424 -0.55140364 -0.26663616\nv -0.04834477 -0.55588579 -0.26470411\nv -0.04802595 -0.55853200 -0.26547131\nv -0.04722165 -0.55730021 -0.26794344\nv 0.00886586 -0.49973521 -0.28690654\nv 0.02382400 -0.49791577 -0.28781819\nv 0.02296103 -0.49095288 -0.28179240\nv 0.00855053 -0.49043617 -0.27936876\nv -0.01485464 -0.47569832 -0.25787091\nv -0.00889027 -0.48091719 -0.26469439\nv -0.00721562 -0.47045276 -0.26337084\nv -0.01862103 -0.46951136 -0.25330690\nv -0.03538990 -0.64407611 -0.19691674\nv -0.04111569 -0.63996148 -0.20285511\nv -0.04090903 -0.63593948 -0.19699973\nv 0.07935655 -0.56315124 -0.22600040\nv 0.08296362 -0.56283033 -0.22376859\nv 0.08355048 -0.56420684 -0.22400731\nv 0.07969001 -0.56444728 -0.22615251\nv 0.09383881 -0.56134856 -0.23176424\nv 0.09341812 -0.56588733 -0.2296342

In [2]:
sys.path.append(os.getcwd())

In [7]:
os.chdir('/home/yuxinyao/GAMMA-release/')

In [10]:
# GAMMA-release/exp_GAMMAPrimitive/vis_GAMMAprimitive.py

In [11]:
import os
import sys
import numpy as np
import open3d as o3d
import torch
import smplx
import cv2
import pickle
import pdb
import re
import glob

# sys.path.append(os.getcwd())
from exp_GAMMAPrimitive.utils.batch_gen_amass import BatchGeneratorAMASSCanonicalized
from exp_GAMMAPrimitive.utils.vislib import *
from exp_GAMMAPrimitive.vis_GAMMAprimitive import *
